# Clasificador Naive Bayes de noticias argentinas

<a target="_blank" href="https://colab.research.google.com/github/pdomins/bayesian-learning/blob/master/ej2_bayes_news.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

El objetivo de este ejercicio es implementar un clasificador de texto utilizando el **clasificador ingenuo de Bayes** sobre el
conjunto de datos *”Noticias Argentinas”* para clasificar cada noticia según su tipo.

Instalaciones previas necesarias:

In [1]:
%%capture
!python -m pip install nltk       # for stopwords
!python -m pip install spacy      # for tokenization
!python -m pip install openpyxl   # for file reading
!python -m spacy download es_core_news_md   # for lemmatization

Librerías utilizadas:

In [2]:
import pandas as pd

import nltk
import spacy

from nltk.corpus import stopwords

## Análisis del dataset

En principio contamos con 164690 tuplas:

In [3]:
df = pd.read_excel("data/Noticias_argentinas_clean.xlsx")
df

,fecha,titular,fuente,categoria
0,2018-12-13 15:49:06,Se van los Melli,NaN,Noticias destacadas
1,2018-12-26 21:21:41,Cantos racistas en el Calcio,NaN,Noticias destacadas
2,2018-12-26 21:21:41,Cantos racistas en el Calcio,NaN,Noticias destacadas
3,2019-01-13 16:35:30,Los que viajan a Uruguay son...,NaN,Noticias destacadas
4,2019-01-13 16:35:30,Los que viajan a Uruguay son...,NaN,Noticias destacadas
...,...,...,...,...
164685,2018-11-26 11:34:11,River Boca: el Gobierno nacional pide â€œinves...,NaN,NaN
164686,2018-11-26 11:34:11,River Boca: el Gobierno nacional pide â€œinves...,NaN,NaN
164687,2018-11-24 22:25:24,Se postergó San Lorenzo Huracán: el resto de l...,NaN,NaN
164688,2018-11-24 22:25:24,Se postergó San Lorenzo Huracán: el resto de l...,NaN,NaN


In [4]:
df['fuente'].unique().shape

(958,)

Al enumerar las categorías nos encontramos con el valor NaN, indicando que hay tuplas sin especificar su categoría:

In [5]:
df['categoria'].unique()

array(['Noticias destacadas', 'Ciencia y Tecnologia', nan, 'Deportes',
       'Entretenimiento', 'Destacadas', 'Actualidad', 'Crítica'],
      dtype=object)

Al contar los valores por cada una:

In [6]:
print(df.groupby('categoria')['titular'].count())

categoria
Actualidad                   1
Ciencia y Tecnologia      2966
Crítica                      4
Deportes                  2969
Destacadas                2971
Entretenimiento           2961
Noticias destacadas     133864
Name: titular, dtype: int64


Además, al observar el dataset más arriba, podemos ver que existen tuplas que se encuentran repetidas. Teniendo esto en cuenta, contamos nuevamente las noticias:

In [7]:
print(df.groupby('categoria')['titular'].nunique())

categoria
Actualidad                  1
Ciencia y Tecnologia      710
Crítica                     1
Deportes                 1402
Destacadas               1731
Entretenimiento          1199
Noticias destacadas     39491
Name: titular, dtype: int64


## Preprocesamiento de los datos

Vamos a expresar los títulos como un array conformado por sus palabras relevantes lematizadas:

In [8]:
# stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

# tokenization and lemmatization
nlp = spacy.load("es_core_news_md")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def preprocess_text(title_string : str):

  # removing unnecesary symbols and tokenization
  title = nlp(title_string.strip('.,+-#:;¿?¡!"\''))

  lemmas = []

  for tok in title:
    word = tok.lemma_.lower()
    if word not in stop_words:
      lemmas.append(word)

  return lemmas

## Armado de vocabulario

Para empezar creamos un diccionario con todas las palabras utilizadas en los titulares para cada categoría, evitando repetidos en cada set:

In [10]:
def get_categories(df : pd.DataFrame):
    return df.dropna(subset=['categoria'])['categoria'].unique()

In [11]:
def get_vocab(df : pd.DataFrame, categories: list):

  vocab = {}

  for category in categories:

    # set of words used in the current category
    cat_vocab = set()

    # subset of training data of the current category
    cat_train_df = df[df['categoria'] == category]

    for title in cat_train_df['titular']:
      cat_vocab.update(title)

    vocab[category] = list(cat_vocab)
  
  return vocab

## Cálculo de frecuencias

Iteramos los titulares dentro de cada categoría y calculamos la frecuencia de aparición de las palabras dentro de cada categoría:

In [12]:
def get_frequencies(df : pd.DataFrame, categories : list, vocab : dict):

  vocab_freq = {}
  laplace_vocab_freq = {}
  titles_by_cat = {}

  for cat_idx, category in enumerate(categories):

    # all the words used the in current category
    word_list = vocab[categories[cat_idx]]

    # dict with the frequencies of every word in the current category
    words_freq = { key: 0 for key in word_list }
    words_laplace_freq = { key: 0 for key in word_list }

    # training data subset of the current category
    cat_train_df = df[df['categoria'] == category]

    # amount of titles in the current category
    cat_title_count = len(cat_train_df['titular'])
    titles_by_cat[category] = cat_title_count

    for word in word_list:
      for title in cat_train_df['titular']:
        if word in title:
          words_freq[word] += (1 / cat_title_count)
          words_laplace_freq[word] += (1 / (cat_title_count+len(categories)))
      words_laplace_freq[word] += (1 / (cat_title_count+len(categories)))

    vocab_freq[category] = words_freq
    laplace_vocab_freq[category] = words_laplace_freq

  return vocab_freq, laplace_vocab_freq, titles_by_cat

## Cálculo de probabilidades

Primero calculamos la probabilidad de que un titular pertenezca a cierta categoría:

In [13]:
def p_cat(categories : list, titles_by_cat : dict):

  title_count = sum(titles_by_cat.values())
  p_cat = { key: 0 for key in categories }

  for category in categories:
    p_cat[category] = titles_by_cat[category] / title_count

  return p_cat

Con esta información y con las frecuencias guardadas en la sección anterior podemos calcular **P(A|categoria)**, es decir, la probabilidad de ocurrencia de un conjunto de palabras dada cierta categoría:

In [14]:
def p_a_cat(conj_a : list, cat_vocab_freq : dict):
  prob = 1
  for word in conj_a:
    prob *= cat_vocab_freq.get(word, 0)
  return prob

In [15]:
def p_a_cat_laplace(conj_a : list, cat_laplace_vocab_freq : dict, total_titles_cat : int, k_classes : int):
  prob = 1
  for word in conj_a:
    freq = cat_laplace_vocab_freq.get(word, 0)
    if freq > 0:
      prob *= freq
    else:
      prob *= (1 / (total_titles_cat+k_classes))
  return prob

Luego obtenemos **P(A)**, la probabilidad de ocurrencia de un conjunto de palabras:

In [16]:
def p_a(conj_a : list, categories : list, prob_cats : dict, vocab_freq : dict, titles_by_cat : dict, laplace_smoothing : bool = False, laplace_vocab_freq : dict = None):
  prob = 0
  for category in categories:
    prob_a_cat = 0
    if laplace_smoothing:
      prob_a_cat = p_a_cat_laplace(conj_a, laplace_vocab_freq[category], titles_by_cat[category], len(categories))
    else:
      prob_a_cat = p_a_cat(conj_a, vocab_freq[category])
    prob += (prob_a_cat * prob_cats[category])
  return prob

Finalmente podemos calcular **P(categoria|A)**, la probabilidad de que un titular pertenezca a cierta categoría dado su conjunto de palabras:

In [17]:
def p_cat_a(prob_a_cat : int, prob_cat : int, prob_a : int):
  return prob_a_cat * prob_cat / prob_a

## Naive Bayes

Calculamos las probabilidades P(categoria|A) para cada categoría y nos quedamos con la de mayor valor:

In [18]:
def naive_bayes(conj_a : list, categories : list, vocab_freq : dict, titles_by_cat : dict, prob_cats : dict, laplace_smoothing : bool = False, laplace_vocab_freq : dict = None):

  p_cats_a = {}
  prob_a = p_a(conj_a, categories, prob_cats, vocab_freq, titles_by_cat, laplace_smoothing, laplace_vocab_freq)

  for category in categories:

    prob_a_cat = 0
    if laplace_smoothing:
      prob_a_cat = p_a_cat_laplace(conj_a, laplace_vocab_freq[category], titles_by_cat[category], len(categories))
    else:
      prob_a_cat = p_a_cat(conj_a, vocab_freq[category])

    p_cats_a[category] = p_cat_a(prob_a_cat, prob_cats[category], prob_a)
  
  return max(p_cats_a, key = lambda k: p_cats_a[k]), p_cats_a

## Probando el clasificador

Utilizamos el método K-Fold para futura cross-validation:

In [19]:
from data_split import k_fold_split

Vamos a crear dos conjuntos, uno de entrenamiento y otro de testeo, por lo que nos queda:

In [20]:
k = 2

test_size = df.shape[0] / k
(test_size * (k-1), test_size)

(82345.0, 82345.0)

In [21]:
train_df, test_df = k_fold_split(df, k)
train_df['titular'] = train_df['titular'].apply(preprocess_text)

In [22]:
categories = get_categories(train_df)
vocab = get_vocab(train_df, categories)
vocab_freq, laplace_vocab_freq, titles_by_cat = get_frequencies(train_df, categories, vocab)
prob_cats = p_cat(categories, titles_by_cat)

In [23]:
selected_class, p_cats_a = naive_bayes(["messi", "pelota"], categories, vocab_freq, titles_by_cat, prob_cats, True, laplace_vocab_freq)

for category in p_cats_a.keys():
    print(f"P ( cat = '{category}' | A ) = {p_cats_a[category]}")

print(f"\nSelected class: {selected_class}")

P ( cat = 'Noticias destacadas' | A ) = 0.6113059800332089
P ( cat = 'Ciencia y Tecnologia' | A ) = 0.0023863669028461256
P ( cat = 'Entretenimiento' | A ) = 0.004614374008064825
P ( cat = 'Deportes' | A ) = 0.2685002945138908
P ( cat = 'Destacadas' | A ) = 0.0023735890696593293
P ( cat = 'Actualidad' | A ) = 0.055409697736165
P ( cat = 'Crítica' | A ) = 0.055409697736165

Selected class: Noticias destacadas


## Error de clasificación

In [27]:
def predict(test_serie : pd.Series):
    title = preprocess_text(test_serie['titular'])
    selected_class, _ = naive_bayes(title, categories, vocab_freq, titles_by_cat, prob_cats, True, laplace_vocab_freq)
    return selected_class

Cross-validation:

In [25]:
from error_functions import compute_classification_error
from df_utils        import get_column_value_dict

In [28]:
test_df_label_dict = get_column_value_dict(test_df, "categoria")
compute_classification_error(test_df, test_df_label_dict, lambda s : predict(s))

81172

In [29]:
test_df.shape

(82345, 4)

## División óptima del conjunto de textos

Próximamente